# Notebook for reproduce the identification of streams in M31 using Pandas:
data @ /home/mardom/Astrophysical-Research/Andromeda-Gaia-DR2/data/pandas \\
code @ /home/mardom/Astrophysical-Research/Andromeda-Gaia-DR2/code/HSS    \\
following https://iopscience.iop.org/article/10.3847/1538-4357/aae8e7/pdf \\
and https://iohttps://iopscience.iop.org/article/10.3847/1538-4357/ac4496/pdf \\
in order to create a library of streams in M31 like those in the MW  and therefore 
contribute to https://github.com/mardom/galstreams.git upstream \\
need: cross correlation with Gaia data by Keiko and  \\
todo: \\
by Mariano Javier de Leon, Dominguez Romero @ IATE-OAC-UNC 2022, mariano.dominguez@unc.edu.ar \\

In [1]:
import regions
import HSS 

import numpy as np
import matplotlib.pyplot as plt

In [2]:
filepath="/media/mardom/_store/mardom/Astrophysical-Research/Andromeda-Gaia-DR2/data/pandas/CATALOGS/"

In [3]:
filename="m001p.ascd.gz"

In [4]:
datasource=filepath+filename

In [5]:
# read pandas fits data at  /home/mardom/Astrophysical-Research/Andromeda-Gaia-DR2/data/pandas
from astropy.io import ascii
data = ascii.read(datasource)  
print(data)                

     RA         Dec      iccd    xg      yg     g      dg   ig    xi      yi     i      di   ii  ia field
----------- ------------ ---- ------- ------- ------ ----- --- ------- ------- ------ ----- --- --- -----
1:40:16.989 +28:00:57.76    1  1336.4 2444.44  13.98 0.001  -9  1331.4 2446.13 13.617 0.001  -9   0     1
1:40:16.292 +27:59:37.16    1  1284.4 2881.92 13.987 0.001  -9 1279.02 2883.41 13.543 0.001  -9   0     1
1:40:22.707 +28:06:54.38    1  1759.3  499.18 14.142 0.001  -9 1755.99  501.99 13.751 0.001  -9   0     1
1:40:18.118 +28:01:38.52    1 1418.71 2222.32 14.157 0.001  -9  1413.9 2224.12 13.668 0.001  -9   0     1
1:40:18.067 +28:03:32.81    1 1418.15 1601.52 14.513 0.001  -9  1413.9 1603.56 13.821 0.001  -9   0     1
1:40:07.711 +28:04:29.12    1  674.08 1304.54  14.59 0.001  -9  670.26 1305.61 14.099 0.001  -9   0     1
1:40:04.072 +27:57:00.08    1  402.73 3740.64 14.683 0.001  -9  396.73 3740.72 14.016 0.001  -9   0     1
1:40:22.857 +28:06:50.88    1  1770.0  518.17 

In [6]:
data.colnames

['RA',
 'Dec',
 'iccd',
 'xg',
 'yg',
 'g',
 'dg',
 'ig',
 'xi',
 'yi',
 'i',
 'di',
 'ii',
 'ia',
 'field']

In [7]:
import glob
import pandas as pd
filepath = "/home/mardom/Astrophysical-Research/Andromeda-Gaia-DR2/data/pandas/CATALOGS/"

dfs = [ascii.read(x) for x in glob.glob(filepath + '*')]

In [8]:
from astropy.table import vstack
from astropy.table import Table
final = Table()
for i in dfs:
    final = vstack([final, i])
    print(i.info, final.info) 
#final_df=final.to_pandas
##final_df = pd.concat(dfs, axis=1, ignore_index=True)

<Table length=154881>
 name  dtype 
----- -------
   RA   str11
  Dec   str12
 iccd   int64
   xg float64
   yg float64
    g float64
   dg float64
   ig   int64
   xi float64
   yi float64
    i float64
   di float64
   ii   int64
   ia   int64
field   int64
 <Table length=154881>
 name  dtype 
----- -------
   RA   str11
  Dec   str12
 iccd   int64
   xg float64
   yg float64
    g float64
   dg float64
   ig   int64
   xi float64
   yi float64
    i float64
   di float64
   ii   int64
   ia   int64
field   int64

<Table length=224359>
 name  dtype 
----- -------
   RA   str11
  Dec   str12
 iccd   int64
   xg float64
   yg float64
    g float64
   dg float64
   ig   int64
   xi float64
   yi float64
    i float64
   di float64
   ii   int64
   ia   int64
field   int64
 <Table length=379240>
 name  dtype 
----- -------
   RA   str11
  Dec   str12
 iccd   int64
   xg float64
   yg float64
    g float64
   dg float64
   ig   int64
   xi float64
   yi float64
    i float64
   di float64

In [9]:
import pickle
with open('mypickle.pickle', 'wb') as f:
    pickle.dump(final, f)

In [10]:
# eliminate duplicates and mask erroneous determinations
condition =  np.logical_or(np.logical_or(final["ig"] == -1, final["ii"] == -1), np.logical_or(final["ig"] == -2, final["ii"] == -2))
stars = final[condition == True]
galax = final[condition == False]
del final

In [ ]:
from astropy.coordinates import Angle
from astropy import units as u

RAV1 = Angle(galax["RA"], u.hour)
galax["RA"] = RAV1.wrap_at(12*u.hour)
galax["Dec"] = Angle(galax["Dec"], u.deg)

from astropy.table import unique
galax_nd = unique(galax, keys=['RA', 'Dec'])
galax_nd

with open('mypandasgalax.pickle', 'wb') as f:
    pickle.dump(galax_nd, f)

In [ ]:
RAV2 = Angle(stars["RA"], u.hour)
stars["RA"] = RAV2.wrap_at(12*u.hour)
stars["Dec"] = Angle(stars["Dec"], u.deg)

stars_nd = unique(stars, keys=['RA', 'Dec'])
stars_nd

with open('mypandasstars.pickle', 'wb') as f:
    pickle.dump(stars_nd, f)

RA,Dec,iccd,xg,yg,g,dg,ig,xi,yi,i,di,ii,ia,field
str12,str12,int64,float64,float64,float64,float64,int64,float64,float64,float64,float64,int64,int64,int64
0:05:06.123,+39:11:38.91,1,48.41,4119.33,17.97,0.001,-9,45.19,4120.71,17.773,0.001,-1,0,197
0:05:32.837,+39:20:35.08,1,1742.52,1193.62,18.217,0.001,-1,1738.98,1196.05,17.471,0.001,-9,0,197
0:05:17.592,+39:20:04.17,1,778.71,1375.23,18.403,0.001,-1,775.4,1377.27,17.847,0.001,-1,0,197
0:05:33.680,+39:20:40.56,1,1796.02,1162.94,18.433,0.001,-1,1792.46,1165.45,17.553,0.001,-9,0,197
0:05:24.232,+39:10:59.02,1,1189.67,4322.99,18.556,0.001,-1,1186.16,4324.82,17.821,0.001,-9,0,197
0:05:23.869,+39:16:39.55,1,1170.81,2480.97,18.554,0.001,-1,1167.36,2483.03,17.607,0.001,-9,0,197
0:05:21.307,+39:14:32.73,1,1007.2,3169.82,18.565,0.001,-1,1003.78,3171.73,17.484,0.001,-9,0,197
0:05:18.730,+39:14:03.44,1,844.21,3330.26,18.586,0.001,-1,840.82,3332.04,16.659,0.001,-9,0,197
0:05:32.031,+39:13:46.25,1,1684.07,3413.34,18.601,0.001,-1,1680.47,3415.46,17.511,0.001,-9,0,197


In [ ]:
plt.scatter(RAV.degree, DEV.degree , c ="blue")
# To show the plot
plt.show()

In [ ]:
#print(final["RA"])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
#image = plt.imread(image_data)
plt.imshow(image_data, cmap='gray')
plt.colorbar()

In [ ]:
print('Min:', np.min(image_data))
print('Max:', np.max(image_data))
print('Mean:', np.mean(image_data))
print('Stdev:', np.std(image_data))

In [ ]:
!more /media/mardom/_store/mardom/Astrophysical-Research/Andromeda-Gaia-DR2/data/pandas/files.txt

In [ ]:
print(type(image_data.flatten()))
print(image_data.flatten().shape)

In [ ]:
histogram = plt.hist(image_data.flatten(), bins='auto')

In [ ]:
from matplotlib.colors import LogNorm

In [ ]:
plt.imshow(image_data, cmap='gray', norm=LogNorm())

# I chose the tick marks based on the histogram above
cbar = plt.colorbar(ticks=[5.e3,1.e4,2.e4])
cbar.ax.set_yticklabels(['5,000','10,000','20,000'])

